# Classifying Urban sounds using Deep Learning

## 4 Model Refinement 

### Load Preprocessed data 

In [1]:
# retrieve the preprocessed data from previous notebook

%store -r x_train 
%store -r x_test 
%store -r y_train 
%store -r y_test 
%store -r yy 
%store -r le

#### Model refinement

In our inital attempt, we were able to achieve a Classification Accuracy score of: 

* Training data Accuracy:  92.3% 
* Testing data Accuracy:  87% 

We will now see if we can improve upon that score using a Convolutional Neural Network (CNN). 

#### Feature Extraction refinement 

In the prevous feature extraction stage, the MFCC vectors would vary in size for the different audio files (depending on the samples duration). 

However, CNNs require a fixed size for all inputs. To overcome this we will zero pad the output vectors to make them all the same size. 

In [2]:
import numpy as np
max_pad_len = 174

def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccs

In [3]:
# Load various imports 
import pandas as pd
import os
import librosa

# Set the path to the full UrbanSound dataset 
fulldatasetpath = '/Users/candre/Documents/UrbanSound8K/audio'

metadata = pd.read_csv('../UrbanSound Dataset sample/metadata/UrbanSound8K.csv')

features = []

# Iterate through each sound file and extract the features 
for index, row in metadata.iterrows():
    
    file_name = os.path.join(os.path.abspath(fulldatasetpath),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    
    class_label = row["class_name"]
    data = extract_features(file_name)
    
    features.append([data, class_label])

# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

print('Finished feature extraction from ', len(featuresdf), ' files') 

/usr/local/lib/python2.7/site-packages/librosa/__init__.py:35: FutureWarning: You are using librosa with Python 2. Please note that librosa 0.7 will be the last version to support Python 2, after which it will require Python 3 or later.
  FutureWarning)


('Finished feature extraction from ', 8732, ' files')


In [4]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

Using TensorFlow backend.


### Convolutional Neural Network (CNN) model architecture 


We will modify our model to be a Convolutional Neural Network (CNN) again using Keras and a Tensorflow backend. 

Again we will use a `sequential` model, starting with a simple model architecture, consisting of four `Conv2D` convolution layers, with our final output layer being a `dense` layer. 

The convolution layers are designed for feature detection. It works by sliding a filter window over the input and performing a matrix multiplication and storing the result in a feature map. This operation is known as a convolution. 


The `filter` parameter specifies the number of nodes in each layer. Each layer will increase in size from 16, 32, 64 to 128, while the `kernel_size` parameter specifies the size of the kernel window which in this case is 2 resulting in a 2x2 filter matrix. 

The first layer will receive the input shape of (40, 174, 1) where 40 is the number of MFCC's 174 is the number of frames taking padding into account and the 1 signifying that the audio is mono. 

The activation function we will be using for our convolutional layers is `ReLU` which is the same as our previous model. We will use a smaller `Dropout` value of 20% on our convolutional layers. 

Each convolutional layer has an associated pooling layer of `MaxPooling2D` type with the final convolutional layer having a `GlobalAveragePooling2D` type. The pooling layer is do reduce the dimensionality of the model (by reducing the parameters and subsquent computation requirements) which serves to shorten the training time and reduce overfitting. The Max Pooling type takes the maximum size for each window and the Global Average Pooling type takes the average which is suitable for feeding into our `dense` output layer.  

Our output layer will have 10 nodes (num_labels) which matches the number of possible classifications. The activation is for our output layer is `softmax`. Softmax makes the output sum up to 1 so the output can be interpreted as probabilities. The model will then make its prediction based on which option has the highest probability.

In [5]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

num_rows = 40
num_columns = 174
num_channels = 1

x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)


print(x_train.size)


num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax')) 

48615600


### Compiling the model 

For compiling our model, we will use the same three parameters as the previous model: 

In [6]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam') 

In [7]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 39, 173, 16)       80        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 19, 86, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 19, 86, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 18, 85, 32)        2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 9, 42, 32)         0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 9, 42, 32)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 41, 64)        

### Training 

Here we will train the model. As training a CNN can take a sigificant amount of time, we will start with a low number of epochs and a low batch size. If we can see from the output that the model is converging, we will increase both numbers.  

In [8]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

#num_epochs = 12
#num_batch_size = 128

num_epochs = 50
num_batch_size = 256

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_cnn.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 6985 samples, validate on 1747 samples
Epoch 1/50
6985/6985 [==============================] - 14s 2ms/step - loss: 3.2422 - accuracy: 0.1921 - val_loss: 2.0417 - val_accuracy: 0.2582

Epoch 00001: val_loss improved from inf to 2.04173, saving model to saved_models/weights.best.basic_cnn.hdf5
Epoch 2/50
6985/6985 [==============================] - 14s 2ms/step - loss: 1.8385 - accuracy: 0.3363 - val_loss: 1.8842 - val_accuracy: 0.3314

Epoch 00002: val_loss improved from 2.04173 to 1.88424, saving model to saved_models/weights.best.basic_cnn.hdf5
Epoch 3/50
6985/6985 [==============================] - 13s 2ms/step - loss: 1.6145 - accuracy: 0.4252 - val_loss: 1.7028 - val_accuracy: 0.4093

Epoch 00003: val_loss improved from 1.88424 to 1.70284, saving model to saved_models/weights.best.basic_cnn.hdf5
Epoch 4/50
6985/6985 [==============================] - 14s 2ms/step - loss: 1.4756 - accuracy: 0.4764 - val_loss: 1.5211 - val_accuracy: 0.4762

Epoch 00004: val_loss improved fr

6985/6985 [==============================] - 14s 2ms/step - loss: 0.5181 - accuracy: 0.8262 - val_loss: 0.5886 - val_accuracy: 0.8208

Epoch 00033: val_loss did not improve from 0.58561
Epoch 34/50
6985/6985 [==============================] - 14s 2ms/step - loss: 0.4817 - accuracy: 0.8395 - val_loss: 0.5271 - val_accuracy: 0.8397

Epoch 00034: val_loss improved from 0.58561 to 0.52715, saving model to saved_models/weights.best.basic_cnn.hdf5
Epoch 35/50
6985/6985 [==============================] - 14s 2ms/step - loss: 0.4886 - accuracy: 0.8349 - val_loss: 0.5855 - val_accuracy: 0.8031

Epoch 00035: val_loss did not improve from 0.52715
Epoch 36/50
6985/6985 [==============================] - 14s 2ms/step - loss: 0.4825 - accuracy: 0.8368 - val_loss: 0.5585 - val_accuracy: 0.8185

Epoch 00036: val_loss did not improve from 0.52715
Epoch 37/50
6985/6985 [==============================] - 14s 2ms/step - loss: 0.4595 - accuracy: 0.8437 - val_loss: 0.5370 - val_accuracy: 0.8277

Epoch 00037

### Test the model 

Here we will review the accuracy of the model on both the training and test data sets. 

In [9]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

('Training Accuracy: ', 0.9148174524307251)
('Testing Accuracy: ', 0.8649112582206726)


The Training and Testing accuracy scores are both high and an increase on our initial model. Training accuracy has increased by ~6% and Testing accuracy has increased by ~4%. 

There is a marginal increase in the difference between the Training and Test scores (~6% compared to ~5% previously) though the difference remains low so the model has not suffered from overfitting. 

### Predictions  

Here we will modify our previous method for testing the models predictions on a specified audio .wav file. 

In [10]:
def print_prediction(file_name):
    prediction_feature = extract_features(file_name) 
    prediction_feature = prediction_feature.reshape(1, num_rows, num_columns, num_channels)

    predicted_vector = model.predict_classes(prediction_feature)
    predicted_class = le.inverse_transform(predicted_vector) 
    print("The predicted class is:", predicted_class[0], '\n') 
    
    predicted_proba_vector = model.predict_proba(prediction_feature) 
    predicted_proba = predicted_proba_vector[0]
    for i in range(len(predicted_proba)): 
        category = le.inverse_transform(np.array([i]))
        print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )

### Validation 

#### Test with sample data 

As before we will verify the predictions using a subsection of the sample audio files we explored in the first notebook. We expect the bulk of these to be classified correctly. 

In [11]:
# Class: Air Conditioner

filename = '../UrbanSound Dataset sample/audio/100852-0-0-0.wav' 
print_prediction(filename) 

('The predicted class is:', 'air_conditioner', '\n')
('air_conditioner', '\t\t : ', '0.99430495500564575195312500000000')
('car_horn', '\t\t : ', '0.00000413229463447351008653640747')
('children_playing', '\t\t : ', '0.00049223139649257063865661621094')
('dog_bark', '\t\t : ', '0.00012356304796412587165832519531')
('drilling', '\t\t : ', '0.00314197433181107044219970703125')
('engine_idling', '\t\t : ', '0.00022964659729041159152984619141')
('gun_shot', '\t\t : ', '0.00017332518473267555236816406250')
('jackhammer', '\t\t : ', '0.00136348372325301170349121093750')
('siren', '\t\t : ', '0.00001274913756788009777665138245')
('street_music', '\t\t : ', '0.00015388085739687085151672363281')


In [12]:
# Class: Drilling

filename = '../UrbanSound Dataset sample/audio/103199-4-0-0.wav'
print_prediction(filename) 

('The predicted class is:', 'drilling', '\n')
('air_conditioner', '\t\t : ', '0.00011117798567283898591995239258')
('car_horn', '\t\t : ', '0.00046746811131015419960021972656')
('children_playing', '\t\t : ', '0.00010200766701018437743186950684')
('dog_bark', '\t\t : ', '0.00002208971636719070374965667725')
('drilling', '\t\t : ', '0.88226091861724853515625000000000')
('engine_idling', '\t\t : ', '0.00139172887429594993591308593750')
('gun_shot', '\t\t : ', '0.00001045745648298179730772972107')
('jackhammer', '\t\t : ', '0.00029792665736749768257141113281')
('siren', '\t\t : ', '0.00000047216988718901120591908693')
('street_music', '\t\t : ', '0.11533586680889129638671875000000')


In [13]:
# Class: Street music 

filename = '../UrbanSound Dataset sample/audio/101848-9-0-0.wav'
print_prediction(filename) 

('The predicted class is:', 'street_music', '\n')
('air_conditioner', '\t\t : ', '0.00020938765374012291431427001953')
('car_horn', '\t\t : ', '0.00030915610841475427150726318359')
('children_playing', '\t\t : ', '0.05565844848752021789550781250000')
('dog_bark', '\t\t : ', '0.00261942483484745025634765625000')
('drilling', '\t\t : ', '0.00005771014548372477293014526367')
('engine_idling', '\t\t : ', '0.00006845247844466939568519592285')
('gun_shot', '\t\t : ', '0.00000000316517190235288126132218')
('jackhammer', '\t\t : ', '0.00000095644850262033287435770035')
('siren', '\t\t : ', '0.00031159777427092194557189941406')
('street_music', '\t\t : ', '0.94076490402221679687500000000000')


In [14]:
# Class: Car Horn 

filename = '../UrbanSound Dataset sample/audio/100648-1-0-0.wav'
print_prediction(filename) 

('The predicted class is:', 'drilling', '\n')
('air_conditioner', '\t\t : ', '0.00413095718249678611755371093750')
('car_horn', '\t\t : ', '0.23555260896682739257812500000000')
('children_playing', '\t\t : ', '0.00341038079932332038879394531250')
('dog_bark', '\t\t : ', '0.07661475241184234619140625000000')
('drilling', '\t\t : ', '0.30554339289665222167968750000000')
('engine_idling', '\t\t : ', '0.01312879007309675216674804687500')
('gun_shot', '\t\t : ', '0.16214396059513092041015625000000')
('jackhammer', '\t\t : ', '0.18499884009361267089843750000000')
('siren', '\t\t : ', '0.00950079876929521560668945312500')
('street_music', '\t\t : ', '0.00497550377622246742248535156250')


#### Observations 

We can see that the model performs well. 

Interestingly, car horn was again incorrectly classifed but this time as drilling - though the per class confidence shows it was a close decision between car horn with 26% confidence and drilling at 34% confidence.  

### Other audio

Again we will further validate our model using a sample of various copyright free sounds that we not part of either our test or training data. 

In [15]:
filename = '../Evaluation audio/dog_bark_1.wav'
print_prediction(filename) 

('The predicted class is:', 'dog_bark', '\n')
('air_conditioner', '\t\t : ', '0.00078123359708115458488464355469')
('car_horn', '\t\t : ', '0.36857819557189941406250000000000')
('children_playing', '\t\t : ', '0.00487326225265860557556152343750')
('dog_bark', '\t\t : ', '0.47308158874511718750000000000000')
('drilling', '\t\t : ', '0.04491152241826057434082031250000')
('engine_idling', '\t\t : ', '0.00239122915081679821014404296875')
('gun_shot', '\t\t : ', '0.09764170646667480468750000000000')
('jackhammer', '\t\t : ', '0.00051994249224662780761718750000')
('siren', '\t\t : ', '0.00198819977231323719024658203125')
('street_music', '\t\t : ', '0.00523302564397454261779785156250')


In [16]:
filename = '../Evaluation audio/drilling_1.wav'

print_prediction(filename) 

('The predicted class is:', 'jackhammer', '\n')
('air_conditioner', '\t\t : ', '0.16449461877346038818359375000000')
('car_horn', '\t\t : ', '0.00005008173684473149478435516357')
('children_playing', '\t\t : ', '0.00067531666718423366546630859375')
('dog_bark', '\t\t : ', '0.00392344594001770019531250000000')
('drilling', '\t\t : ', '0.02171051502227783203125000000000')
('engine_idling', '\t\t : ', '0.01464555878192186355590820312500')
('gun_shot', '\t\t : ', '0.00003229510548408143222332000732')
('jackhammer', '\t\t : ', '0.79288184642791748046875000000000')
('siren', '\t\t : ', '0.00108002603519707918167114257812')
('street_music', '\t\t : ', '0.00050631625344976782798767089844')


In [17]:
filename = '../Evaluation audio/gun_shot_1.wav'

print_prediction(filename) 

('The predicted class is:', 'gun_shot', '\n')
('air_conditioner', '\t\t : ', '0.00000130929652186750900000333786')
('car_horn', '\t\t : ', '0.00001913116102514322847127914429')
('children_playing', '\t\t : ', '0.00365575496107339859008789062500')
('dog_bark', '\t\t : ', '0.06901239603757858276367187500000')
('drilling', '\t\t : ', '0.00368810654617846012115478515625')
('engine_idling', '\t\t : ', '0.00033480214187875390052795410156')
('gun_shot', '\t\t : ', '0.92230415344238281250000000000000')
('jackhammer', '\t\t : ', '0.00000770927272242261096835136414')
('siren', '\t\t : ', '0.00055570778204128146171569824219')
('street_music', '\t\t : ', '0.00042099214624613523483276367188')


In [20]:
import pickle
pickle_out = open("CNN_model.pickle","wb")
pickle.dump(model, pickle_out)
pickle_out.close()

#### Observations 

The performance of our final model is very good and has generalised well, seeming to predict well when tested against new audio data. 

In [21]:
print(model)

In [22]:
pickle_out2 = open("le.pickle","wb")
pickle.dump(le, pickle_out2)
pickle_out.close()